In [2]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\Working_attributes")
from Imports import * 

In [98]:
print_excel = True

In [27]:
# export_from_WHWEEK(
query = f'''with table_1 as (
select "supplier_company_id","mediator_sales_unit","trader_sales_unit","sales_model","sales_unit","customer_company","to_customers_seller_id", "##inserted", "deal_id", trunc("##inserted") as "reportDate"
from "XXWH"."flowDealsReportChanges" 
where "deal_state" = 'Confirmed'
and "##deleted" = 'N'
and "is_closed" = 'No'
--fetch first 10 rows only
)
select * from table_1
where "reportDate" = TO_DATE('2024-08-31', 'YYYY-MM-DD')
'''
data_XXWH = export_from_WHWEEK(query)
data_XXWH.shape

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:176: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


(543, 10)

In [95]:
# исключаем строки
# 1 условие
data_XXWH['customer_buCode'] = merge_Mapping(data_XXWH, 'customer_company')
con_1 = data_XXWH[(data_XXWH.supplier_company_id.isna())&(data_XXWH.mediator_sales_unit.isna())&(data_XXWH.trader_sales_unit.isna())]
indexes_list_1 = con_1[con_1.sales_model=='Replenishment'].index.tolist()
indexes_list_2 = con_1[con_1.customer_buCode==con_1.sales_unit].index.tolist()
# indexes_list_2 = con_1[con_1.to_customers_seller_id==con_1.sales_unit].index.tolist()
indexes_list = list(set(indexes_list_1+indexes_list_2))
data_XXWH_work = data_XXWH[~data_XXWH.index.isin(indexes_list)].reset_index(drop=True)
# 2 условие
data_XXWH_work = data_XXWH_work[~data_XXWH_work.customer_buCode.isin(['BRFTO', 'BRHRG'])]
# последняя запись на дату сделки
data_XXWH_work['max_inserted'] = data_XXWH_work.groupby(dropna=False, as_index=False, by='deal_id')['##inserted'].transform('max')
data_XXWH_work = data_XXWH_work[data_XXWH_work['##inserted']==data_XXWH_work.max_inserted]
data_XXWH_work.shape

(114, 12)

In [96]:
# достаем список строк
deals_list_BD = data_XXWH_work.deal_id.unique().tolist()
len(deals_list_BD)

114

In [77]:
query = f'''
select *
from "RISKACCESS"."unifiedOcp"
where "sourceTable" = 'flowDealsReportChanges'
and "reportDate" = TO_DATE('2024-08-31', 'YYYY-MM-DD')
--fetch first 1 rows only
'''
data_OCP = export_from_RISKCUSTOM(query)
data_OCP.shape

(4791, 40)

In [89]:
data_OCP_deals_list = data_OCP.recordId.unique().astype(int).tolist()
len(data_OCP_deals_list)

1953

In [104]:
list_not_in_OCP = [x for x in deals_list_BD if x not in data_OCP_deals_list]
list_in_OCP = [x for x in deals_list_BD if x in data_OCP_deals_list]

In [105]:
# to excel
if print_excel == True:
    Output_file = f'{str(date.today())}_Deals_recodID_vs_OCP.xlsx'
    with pd.ExcelWriter(Output_file, engine='openpyxl') as writer:
        data_XXWH_work.to_excel(writer, index=False, sheet_name='data_XXWH')
        data_OCP.to_excel(writer, index=False, sheet_name='data_unOCP')
        pd.DataFrame({'XXWH_deals':deals_list_BD}).to_excel(writer, index=False, sheet_name='deals_list', startcol=0)
        pd.DataFrame({'unOCP_deals':data_OCP_deals_list}).to_excel(writer, index=False, sheet_name='deals_list', startcol=2)
        pd.DataFrame({'deals_not_in_unOCP':list_not_in_OCP}).to_excel(writer, index=False, sheet_name='deals_list', startcol=5)
        pd.DataFrame({'deals_in_unOCP':list_in_OCP}).to_excel(writer, index=False, sheet_name='deals_list', startcol=7)